### Melakukan import library yang dibutuhkan

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

### Membaca dataset training dan testing

In [2]:
df = pd.read_csv("isaPerson.csv")

In [3]:
df.shape

(3705, 29)

In [4]:
td = pd.read_csv("tdsd.tsv", sep="\t")

In [5]:
df.head()

,contributors_enabled,created_at,default_profile,default_profile_image,description,entities,favourites_count,followers_count,friends_count,geo_enabled,...,profile_use_background_image,protected,screen_name,statuses_count,url,verified,username,isaPerson,userLocation,jobArea
0,False,08/02/2015 0:32,True,False,NaN,NaN,1.0,16858.0,6.0,False,...,True,False,otodriver,17695.0,NaN,False,@otodriver,nonperson,jabodetabek,hobi
1,False,07/07/2009 13:13,False,False,Markas Pusat Palang Merah Indonesia | Jalan Je...,NaN,2247.0,1033888.0,1235.0,True,...,True,False,palangmerah,53538.0,https://t.co/mx3e3UYXPi,True,@palangmerah,nonperson,jabodetabek,kesehatan
2,False,16/11/2011 10:36,False,False,Ada diskon setiap hari cek di aplikasi #Pegipe...,NaN,1439.0,46217.0,36.0,True,...,True,False,pegipegi,35329.0,https://t.co/lP16dITceB,True,@pegipegi,nonperson,jabodetabek,hospitality & tourism
3,False,13/04/2008 03:30,False,False,#Ngahiji Menang Bersama! Download #PERSIBApp ð...,NaN,1464.0,3502961.0,90.0,True,...,True,False,persib,46556.0,https://t.co/hqHLwIrxfY,True,@persib,nonperson,jawa barat dan banten,olahraga
4,False,19/09/2014 03:05,False,False,Akun Resmi | Dikelola Subbaghumas | Call Cente...,NaN,316.0,26707.0,204.0,True,...,False,False,polresjogja,9204.0,https://t.co/XrZB8YcypN,True,@polresjogja,nonperson,jawa tengah dan yogyakarta,sosial kemasyarakatan


In [6]:
df.columns

Index(['contributors_enabled', 'created_at', 'default_profile',
       'default_profile_image', 'description', 'entities', 'favourites_count',
       'followers_count', 'friends_count', 'geo_enabled', 'id_str', 'name',
       'notifications', 'profile_background_color', 'profile_link_color',
       'profile_location', 'profile_sidebar_border_color',
       'profile_sidebar_fill_color', 'profile_text_color',
       'profile_use_background_image', 'protected', 'screen_name',
       'statuses_count', 'url', 'verified', 'username', 'isaPerson',
       'userLocation', 'jobArea'],
      dtype='object')

In [7]:
tcsv = pd.DataFrame(index=td.index.copy(), columns=[0, 1])

### Melakukan assignment kembali ke dataframe untuk kolom-kolom yang akan digunakan (memilih kolom)

In [8]:
used_column = ['favourites_count', 'statuses_count', 'geo_enabled', 'profile_location', 'followers_count', 'friends_count', 
               'isaPerson', 'verified', 'url', 'protected', 'default_profile_image']
df = df[used_column]
df.head()
td = td[used_column]
td.head()

,favourites_count,statuses_count,geo_enabled,profile_location,followers_count,friends_count,isaPerson,verified,url,protected,default_profile_image
0,153.0,6993.0,True,NaN,146.0,257.0,NaN,False,NaN,False,False
1,1957.0,32272.0,True,NaN,765907.0,348.0,NaN,True,https://t.co/qqFyZ8MBI3,False,False
2,22.0,5285.0,True,NaN,34867.0,22.0,NaN,False,https://t.co/I43yb1p3Pm,False,False
3,3825.0,11815.0,True,NaN,310.0,1130.0,NaN,False,NaN,False,False
4,7.0,29730.0,True,NaN,347.0,316.0,NaN,False,NaN,False,False


### Melakukan preprocessing data dengan mengubah data yang kosong menjadi 0 atau ""

In [9]:
for i in used_column:
    print(df[i].isna().sum())
    print(td[i].isna().sum())

7
6
7
8
7
6
3415
3883
7
6
7
6
0
4153
3
9
1860
2013
7
8
5
6


In [10]:
td = td.apply(lambda x: x.fillna(0) if x.dtype.kind in 'biufc' else x.fillna(''))
df = df.apply(lambda x: x.fillna(0) if x.dtype.kind in 'biufc' else x.fillna(''))

In [11]:
cleanup = {"geo_enabled":{True: 2, False: 1, "": 0},
           "isaPerson":{"person": 1, "nonperson": 0},
           "protected":{True: 2, False: 1, "": 0},
           "verified":{True: 2, False: 1, "": 0},
           "default_profile_image":{True: 2, False: 1, "": 0},
          }
td = td.replace(cleanup)
df = df.replace(cleanup)

In [12]:
df['profile_location'] = df['profile_location'].apply(lambda x:1 if x != '' else 0)
df['url'] = df['url'].apply(lambda x:1 if x != '' else 0)
td['profile_location'] = td['profile_location'].apply(lambda x:1 if x != '' else 0)
td['url'] = td['url'].apply(lambda x:1 if x != '' else 0)

In [13]:
for i in used_column:
    df[i] = df[i].astype(int)
    td[i] = td[i].astype(int)

In [14]:
print(df.dtypes)
print(td.dtypes)

favourites_count         int32
statuses_count           int32
geo_enabled              int32
profile_location         int32
followers_count          int32
friends_count            int32
isaPerson                int32
verified                 int32
url                      int32
protected                int32
default_profile_image    int32
dtype: object
favourites_count         int32
statuses_count           int32
geo_enabled              int32
profile_location         int32
followers_count          int32
friends_count            int32
isaPerson                int32
verified                 int32
url                      int32
protected                int32
default_profile_image    int32
dtype: object


### Normalisasi data dalam range 0-1

In [15]:
df -= df.min()
df //= (df.max() - df.min())
df.head()

td -= td.min()
td //= (td.max() - td.min())
td.head()

,favourites_count,statuses_count,geo_enabled,profile_location,followers_count,friends_count,isaPerson,verified,url,protected,default_profile_image
0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,NaN,1.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0


In [16]:
print(df.corr())

                       favourites_count  statuses_count  geo_enabled  \
favourites_count               1.000000       -0.000270     0.014699   
statuses_count                -0.000270        1.000000     0.014699   
geo_enabled                    0.014699        0.014699     1.000000   
profile_location              -0.004788       -0.004788    -0.028483   
followers_count               -0.000270       -0.000270     0.014699   
friends_count                 -0.000270       -0.000270     0.014699   
isaPerson                      0.010864       -0.024851    -0.047214   
verified                       0.043914        0.043914     0.139447   
url                            0.016498        0.016498     0.176167   
protected                     -0.008014       -0.008014    -0.015842   
default_profile_image         -0.002196       -0.002196    -0.136965   

                       profile_location  followers_count  friends_count  \
favourites_count              -0.004788        -0.000270    

### Mencari algoritma terbaik

In [17]:
cba = ['profile_location', 'default_profile_image', 'followers_count', 'protected', 
       'verified', 'url']

X_train, X_test, y_train, y_test = train_test_split(df[cba], df.isaPerson, test_size=0.6, random_state=1)

In [18]:
dt = DecisionTreeClassifier(random_state=1)
rf = RandomForestClassifier(random_state=1)
gb = GaussianNB()
mb = MultinomialNB()
lr = LogisticRegression(random_state=1)
svc = make_pipeline(StandardScaler(), SVC(gamma='auto'))
sgc = make_pipeline(StandardScaler(), SGDClassifier(max_iter=1000, tol=1e-3))
knn = KNeighborsClassifier(n_neighbors=2)
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=1, random_state=1)
lm = [dt, rf, gb, mb, lr, svc, sgc, knn, gbc]

In [19]:
for i in lm:
    i.fit(X_train, y_train)
    pred = i.predict(X_test)
    average = (accuracy_score(y_test, pred) + precision_score(y_test, pred) + recall_score(y_test, pred) + 
               f1_score(y_test,pred))/4
    print(i)
    print("Akurasi = ", accuracy_score(y_test, pred))
    print("Precision = ", precision_score(y_test, pred))
    print("Recall = ", recall_score(y_test, pred))
    print("F1 = ", f1_score(y_test,pred))
    print("Average = ", average)
    print("================================")

DecisionTreeClassifier(random_state=1)
Akurasi =  0.7377417903733693
Precision =  0.75498426023085
Recall =  0.9254019292604502
F1 =  0.8315515746893962
Average =  0.8124198886385164
RandomForestClassifier(random_state=1)
Akurasi =  0.7377417903733693
Precision =  0.7547169811320755
Recall =  0.9260450160771704
F1 =  0.8316488593704882
Average =  0.8125381617382759
GaussianNB()
Akurasi =  0.7017543859649122
Precision =  0.900359066427289
Recall =  0.645016077170418
F1 =  0.751592356687898
Average =  0.7496804715626293
MultinomialNB()
Akurasi =  0.7246963562753036
Precision =  0.7382516422435573
Recall =  0.9395498392282958
F1 =  0.8268251273344652
Average =  0.8073307412704055
LogisticRegression(random_state=1)
Akurasi =  0.7377417903733693
Precision =  0.7547169811320755
Recall =  0.9260450160771704
F1 =  0.8316488593704882
Average =  0.8125381617382759
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])
Akurasi =  0.7377417903733693
Prec

In [20]:
tcsv = pd.DataFrame(index=td.index.copy(), columns=['id', 'person'])
tcsv[0] = td.index
tcsv[1]= td['isaPerson']

In [21]:
tcsv.to_csv("pred.csv".format(2), index=False)

### Melakukan proses prediksi pada data testing

In [22]:
td.shape

(4153, 11)

In [23]:
# nanti digabung aja dua list ini biar ga repot, samain header di features.
features = ['url']
train_X = df[features]
train_y = df.isaPerson

lr = RandomForestClassifier(random_state=1) #(random_state=1)
lr.fit(train_X,train_y)
pred = lr.predict(td[features])

td['isaPerson'] = pred

In [24]:
td[features].head()

,url
0,0.0
1,1.0
2,1.0
3,0.0
4,0.0


In [25]:
tcsv = pd.DataFrame(index=td.index.copy(), columns=[0, 1])
tcsv[0] = td.index
tcsv[1]= td['isaPerson']

In [26]:
tcsv.head(10)

,0,1
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
5,5,1
6,6,1
7,7,1
8,8,1
9,9,1


In [27]:
td = pd.read_csv("tdsd.tsv", sep="\t")

### Overwrite hasil prediksi sebelumnya apabila description dan username mengandung kata tertentu

In [28]:
td['description'] = td['description'].fillna('')
td['username'] = td['username'].fillna('')
for i in range(4153):
    row = td.loc[i]
    if(row['description'].lower().find("official") != -1):
        tcsv.loc[i, 1] = 0
    elif(row['description'].lower().find("akun resmi") != -1):
        tcsv.loc[i, 1] = 0
    elif(row['description'].lower().find("akun twitter resmi") != -1):
        tcsv.loc[i, 1] = 0
    elif(row['username'].lower().find("official") != -1):
        tcsv.loc[i, 1] = 0

In [29]:
tcsv.head(10)

,0,1
0,0,1
1,1,0
2,2,0
3,3,1
4,4,1
5,5,1
6,6,1
7,7,1
8,8,0
9,9,1


### Mengeluarkan result dalam bentuk csv

In [30]:
tcsv.to_csv("submission5.csv".format(2), index=False)